In [ ]:
%load_ext autoreload
%autoreload 2

from common import *

graphics_res = Resources["graphics"]
object_registry = Resources.special('objects.reg').content

In [ ]:
scenario_res = Resources["scenario"]
world_res = Resources["world"]
main_res = Resources["main"]

In [ ]:
graphics_res = Resources['graphics']
unit_registry = Resources.special("units.reg").content
# 64 goblin
# 76 turtle

In [ ]:
databin = Resources.special("data.bin").content
troll_id = databin.monsters_by_name["troll"].type_id
troll_record = unit_registry.units_by_id[troll_id]

unit256 = Resources["graphics", "units", troll_record['filename'] + ".256"].content
unit256[1].to_pil_image()

In [ ]:
for path, node in unit_registry.root_descent():
    pass
    # print(path, node.name, node.value.value)

In [ ]:
graphics_res = Resources["graphics"]
object_registry = graphics_res[jn('objects','objects.reg')].content

In [ ]:
class ObjectsDump:
    name = 'objects'
    
    def __init__(self):        
        object_registry = Resources.special('objects.reg').content
        self.objects_by_id = object_registry.objects_by_id
    
    def data(self):    
        header = ['id', 'desc', 'sprite_id', 'sprite', 'width','height','centerx','centery','image']
        yield header
        fields = ["id", "desctext", "index", "filename", "width", "height", "centerx", "centery"]
        for oid, obj_record in self.objects_by_id.items():        
            obj_file = obj_record['filename']
            if 'fire' in obj_file:
                continue                        
            yield [obj_record[field] for field in fields]
            
    def sprites(self):                         
        for oid, obj_record in self.objects_by_id.items():        
            obj_file = obj_record['filename']
            if 'fire' in obj_file:
                continue
            a256 = Resources["graphics", "objects", obj_record['filename'] + '.256'].content
            index = obj_record['index']
            image = a256[index].to_pil_image()
            yield image        
    
    @classmethod
    def dump(cls, output_filename):
        import openpyxl
        from src import xlsx

        wb = openpyxl.Workbook()
        ws = wb.active

        sheet = ObjectsSheet()

        for row in sheet.data():        
            ws.append(row)

        #autofit columns
        for col in ws.columns:    
            max_width = max([len(str(cell.value)) for cell in col])
            letter = col[0].column_letter
            ws.column_dimensions[letter].width = 1 + max_width

        xlsx.add_image_column(ws, sheet.sprites(), len(row) + 1)

        # Save the file
        wb.save(output_filename)
        wb.close()
        print('done')

In [ ]:
objs = ObjectsDump()
#for im in objs.sprites():
    #print(im.width)
    
plot.images_rect(list(objs.sprites()))

In [ ]:
# ObjectsDump.dump('objects_reg.xlsx')

In [ ]:
structure_registry = Resources.special("structures.reg").content
structure_registry.structures[1]

In [ ]:
class StructuresDump:
    name = 'structures'    
    
    def __init__(self):        
        structure_registry = Resources.special('structures.reg').content
        self.structures = structure_registry.structures
    
    def data(self):    
        header = ['id', 'desc', 'sprite_name', 'sprite', 'picture_name', 'picture']
        yield header                
        for obj in self.structures:
            yield [obj['id'], obj['desctext'], obj['file'], '', obj['picture'], '']
            
    def sprites(self):                
        for obj in self.structures:
            a256 = Resources["graphics", "structures", obj["filename"] + ".256"].content
            w, h = obj['tilewidth'], obj['fullheight']                               
            image = a256.to_tiles(w, h)
            image = PIL.Image.fromarray(image)
            yield image
            
    def pictures(self):        
        import PIL
        for obj in self.structures:                      
            obj_file = obj['picture']
            
            res_path = jn('infowindow', obj['picture'] + '.bmp')
            try:
                bmp = Resources["graphics", res_path].content
                yield bmp
            except KeyError:                
                print(f"Warning: {res_path} is not found")
                yield None       
    
    @classmethod
    def dump(cls, output_filename):
        import openpyxl

        wb = openpyxl.Workbook()
        ws = wb.active

        sheet = cls()

        for row in sheet.data():        
            ws.append(row)

        #autofit columns
        for col in ws.columns:    
            max_width = max([len(str(cell.value)) for cell in col])
            letter = col[0].column_letter
            ws.column_dimensions[letter].width = 1 + max_width

        xlsx.add_image_column(ws, sheet.sprites(), 4)
        xlsx.add_image_column(ws, sheet.pictures(), 6)
    
        # Save the file
        wb.save(output_filename)
        wb.close()
        print('done')

In [ ]:
# StructuresDump.dump('structures_reg.xlsx')

In [ ]:
from src.formats import registry
jn = os.path.join
npc_registry = Resources['scenario','npc.reg'].content
for path, node in npc_registry.root_descent():
    print(path, node.name, node.value.value)

In [ ]:
ai_registry = Resources['world', 'data', 'ai.reg'].content
for path, node in ai_registry.root_descent():
    print(path, node.name, node.value.value)

In [ ]:
map_registry = Resources['world', 'data', 'map.reg'].content
for path, node in map_registry.root_descent():
    print(path, node.name, node.value.value)